In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from preprocessing import *

In [2]:
# Loading Data
data_raw = pd.read_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/mxmh_survey_results.csv')

In [3]:
# Step 1: Dropping Features/Samples
    # Features
cols_drop = ['Permissions', 'Music effects', 'Timestamp', 'BPM']
drop_features = drop_irreleveant(data_raw, cols_drop)

    # Samples --> w/ NaN
drop_samples = drop_features.dropna()

In [4]:
# Step 2: Convert Mental Illness Scores to Categorical
features_range = ['Anxiety', 'Depression', 'OCD', 'Insomnia']
data_ranged = convert_ranges_binary(drop_samples, features_range)

/Users/JasonLebov_1/Desktop/CS 470/Final Project/CS470_FinalProject_Code/preprocessing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = pd.cut(df[feature],
/Users/JasonLebov_1/Desktop/CS 470/Final Project/CS470_FinalProject_Code/preprocessing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = pd.cut(df[feature],
/Users/JasonLebov_1/Desktop/CS 470/Final Project/CS470_FinalProject_Code/preprocessing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [5]:
# Step 3: One Hot Encode Features
cols_dummy = ["Primary streaming service", "Fav genre"]
one_hot_df = one_hot_encode(data_ranged, cols_dummy)

In [6]:
# Step 4: Label Encoding Features
encode_features_binary = ["While working" , "Instrumentalist", "Composer", "Exploratory", "Foreign languages"]
binary_options = ['No', 'Yes']
label_encoded_df = label_encode(one_hot_df, encode_features_binary, binary_options)

In [7]:
# Step 4 [cont]: Label Encoding Features
encode_features_cat = ["Frequency [Classical]", "Frequency [Country]", "Frequency [EDM]", "Frequency [Folk]", 'Frequency [Gospel]', 
                   "Frequency [Hip hop]", "Frequency [Jazz]", "Frequency [K pop]", "Frequency [Latin]", "Frequency [Lofi]", "Frequency [Metal]", "Frequency [Pop]","Frequency [R&B]", "Frequency [Rap]",'Frequency [Rock]', 
                   'Frequency [Video game music]']
cat_options = ['Never', 'Rarely', 'Sometimes', 'Very frequently']
label_encoded_df_final = label_encode(label_encoded_df, encode_features_cat, cat_options)

In [8]:
# Move Label's to End of DF
cols = ['Anxiety', 'Depression', 'OCD', 'Insomnia']
numeric_df_final = label_end(label_encoded_df_final, cols)

In [9]:
# Step 5: Pearson Correlation Matrix - Features
cols_left_out = ['Anxiety', 'OCD', 'Insomnia', 'Depression']
feature_isolate = numeric_df_final.drop(columns=cols_left_out, axis=1)
feature_corr = pearson_matrix(feature_isolate)

In [10]:
# Step 5 [cont]: Dropping Highly Correlated Features --> Threshold (delta) = 0.75
corr_feature_drop = pearson_drop_delta(numeric_df_final, feature_corr, 0.75)

['Frequency [Hip hop]']


In [11]:
# Step 6: Creating 4 Datasets, for Each Illness
anxiety_df = corr_feature_drop.drop(columns=['Depression', 'OCD', 'Insomnia'], axis=1)
depression_df = corr_feature_drop.drop(columns=['Anxiety', 'OCD', 'Insomnia'], axis=1)
ocd_df = corr_feature_drop.drop(columns=['Anxiety', 'Depression', 'Insomnia'], axis=1)
insomnia_df = corr_feature_drop.drop(columns=['Anxiety', 'Depression', 'OCD'], axis=1)

In [12]:
# Step 6 [cont]: Removing 'Symptoms_' Prefix from Label Column
anxiety_df = remove_prefix(anxiety_df, 'Anxiety')
depression_df = remove_prefix(depression_df, 'Depression')
ocd_df = remove_prefix(ocd_df, 'OCD')
insomnia_df = remove_prefix(insomnia_df, 'Insomnia')

In [13]:
# Step 7: Label Encoding Label Feature
feature_options = ['No', 'Yes']

anxiety_encoded = label_encode(anxiety_df, ['Anxiety'], feature_options)
depression_encoded = label_encode(depression_df, ['Depression'], feature_options)
ocd_encoded = label_encode(ocd_df, ['OCD'], feature_options)
insomnia_encoded = label_encode(insomnia_df, ['Insomnia'], feature_options)

In [14]:
# Step 8: Rerun of Pearson Correlation for Labels

anxiety_corr = pearson_matrix(anxiety_encoded)
depression_corr = pearson_matrix(depression_encoded)
ocd_corr = pearson_matrix(ocd_encoded)
insomnia_corr = pearson_matrix(insomnia_encoded)

In [15]:
# Step 8 [cont]: Dropping Highly Correlated Features to Label --> Threshold (gamma) = 0.01
anxiety_drop = pearson_drop_gamma(anxiety_encoded, anxiety_corr, 0.01)
depression_drop = pearson_drop_gamma(depression_encoded, depression_corr, 0.01)
ocd_drop = pearson_drop_gamma(ocd_encoded, ocd_corr, 0.01)
insomnia_drop = pearson_drop_gamma(insomnia_encoded, insomnia_corr, 0.01)

features_to_drop = set(anxiety_drop).intersection(depression_drop, ocd_drop, insomnia_drop)

['Fav genre_Metal', 'Fav genre_Jazz', 'Fav genre_Lofi', 'Composer']
['Frequency [K pop]', 'Instrumentalist', 'Frequency [Classical]', 'Fav genre_Folk']
['Fav genre_Jazz', 'Frequency [Latin]', 'Fav genre_R&B', 'Fav genre_Country', 'Foreign languages', 'Frequency [Metal]', 'Frequency [Jazz]', 'Fav genre_Rock', 'Fav genre_K pop', 'Primary streaming service_Apple Music', 'Fav genre_Hip hop']
['Primary streaming service_Other streaming service', 'Frequency [Gospel]', 'Fav genre_Jazz', 'Frequency [Pop]', 'Age', 'Fav genre_Folk', 'Fav genre_Latin', 'Fav genre_K pop']


In [16]:
features_to_drop

set()

In [17]:
anxiety_final_full = drop_irreleveant(anxiety_encoded, features_to_drop)
depression_final_full = drop_irreleveant(depression_encoded, features_to_drop)
ocd_final_full = drop_irreleveant(ocd_encoded, features_to_drop)
insomnia_final_full = drop_irreleveant(insomnia_encoded, features_to_drop)

In [18]:
# Step 9: Create Train-Test-Splits of Data
anxiety_train, anxiety_test = data_split(anxiety_final_full)
depression_train, depression_test = data_split(depression_final_full)
ocd_train, ocd_test = data_split(ocd_final_full)
insomnia_train, insomnia_test = data_split(insomnia_final_full)

In [19]:
# VISUALIZE THESE
anxiety_train['Anxiety'].value_counts()

Anxiety
1    405
0    175
Name: count, dtype: int64

In [20]:
depression_train['Depression'].value_counts()

Depression
1    317
0    263
Name: count, dtype: int64

In [21]:
ocd_train['OCD'].value_counts()

OCD
0    438
1    142
Name: count, dtype: int64

In [22]:
insomnia_train['Insomnia'].value_counts()

Insomnia
0    353
1    227
Name: count, dtype: int64

In [23]:
display(anxiety_final_full.shape)
display(depression_final_full.shape)
display(ocd_final_full.shape)
display(insomnia_final_full.shape)

(725, 45)

(725, 45)

(725, 45)

(725, 45)

In [24]:
# Step 10: Min-Max Scaling --> Only on Training Data              
cols_scale = anxiety_final_full.iloc[:, :-1].columns
anxiety_train_final, anxiety_test_final = min_max_scale(anxiety_train, anxiety_test, cols_scale)
depression_train_final, depression_test_final = min_max_scale(depression_train, depression_test, cols_scale)
ocd_train_final, ocd_test_final = min_max_scale(ocd_train, ocd_test, cols_scale)
insomnia_train_final, insomnia_test_final = min_max_scale(insomnia_train, insomnia_test, cols_scale)

In [25]:
# Making xFeat & y

    # Anxiety
anxiety_train_xFeat = anxiety_train_final.iloc[:, :-1]
anxiety_train_y = anxiety_train_final.iloc[:, -1]
anxiety_test_xFeat = anxiety_test_final.iloc[:, :-1]
anxiety_test_y = anxiety_test_final.iloc[:, -1]

    # Depression
depression_train_xFeat = depression_train_final.iloc[:, :-1]
depression_train_y = depression_train_final.iloc[:, -1]
depression_test_xFeat = depression_test_final.iloc[:, :-1]
depression_test_y = depression_test_final.iloc[:, -1]

    # OCD
ocd_train_xFeat = ocd_train_final.iloc[:, :-1]
ocd_train_y = ocd_train_final.iloc[:, -1]
ocd_test_xFeat = ocd_test_final.iloc[:, :-1]
ocd_test_y = ocd_test_final.iloc[:, -1]

    # Insomnia
insomnia_train_xFeat = insomnia_train_final.iloc[:, :-1]
insomnia_train_y = insomnia_train_final.iloc[:, -1]
insomnia_test_xFeat = insomnia_test_final.iloc[:, :-1]
insomnia_test_y = insomnia_test_final.iloc[:, -1]


In [26]:
# Applying SMOTE to Training Data

    # Anxiety
final_anxiety_train_xFeat, final_anxiety_train_y = smote(anxiety_train_xFeat, anxiety_train_y)

    # Depression
final_depression_train_xFeat, final_depression_train_y = smote(depression_train_xFeat, depression_train_y)

    # OCD
final_ocd_train_xFeat, final_ocd_train_y = smote(ocd_train_xFeat, ocd_train_y)

    # Insiomnia
final_insomnia_train_xFeat, final_insomnia_train_y = smote(insomnia_train_xFeat, insomnia_train_y)

In [27]:
display(final_anxiety_train_xFeat.shape)
display(final_anxiety_train_y.shape)
display(final_depression_train_xFeat.shape)
display(final_depression_train_y.shape)
display(final_ocd_train_xFeat.shape)
display(final_ocd_train_y.shape)
display(final_insomnia_train_xFeat.shape)
display(final_insomnia_train_y.shape)

(810, 44)

(810,)

(634, 44)

(634,)

(876, 44)

(876,)

(706, 44)

(706,)

In [28]:
display(final_anxiety_train_y.value_counts())
display(final_depression_train_y.value_counts())
display(final_ocd_train_y.value_counts())
display(final_insomnia_train_y.value_counts())

Anxiety
1    405
0    405
Name: count, dtype: int64

Depression
1    317
0    317
Name: count, dtype: int64

OCD
1    438
0    438
Name: count, dtype: int64

Insomnia
0    353
1    353
Name: count, dtype: int64

In [29]:
# Step 11: Save Files to CSV -- CHANGE FILE PATHS

    # Save ' '_final_full
anxiety_final_full.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_anxiety_final_full.csv', index=False)
depression_final_full.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_depression_final_full.csv', index=False)
ocd_final_full.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_ocd_final_full.csv', index=False)
insomnia_final_full.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_insomnia_final_full.csv', index=False)

    # Save Train-Test-Splits
final_anxiety_train_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_anxiety_train_xFeat.csv', index=False)
final_anxiety_train_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_anxiety_train_y.csv', index=False)
anxiety_test_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_anxiety_test_xFeat.csv', index=False)
anxiety_test_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_anxiety_test_y.csv', index=False)

final_depression_train_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_depression_train_xFeat.csv', index=False)
final_depression_train_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_depression_train_y.csv', index=False)
depression_test_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_depression_test_xFeat.csv', index=False)
depression_test_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_depression_test_y.csv', index=False)

final_ocd_train_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_ocd_train_xFeat.csv', index=False)
final_ocd_train_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_ocd_train_y.csv', index=False)
ocd_test_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_ocd_test_xFeat.csv', index=False)
ocd_test_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_ocd_test_y.csv', index=False)

final_insomnia_train_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_insomnia_train_xFeat.csv', index=False)
final_insomnia_train_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_insomnia_train_y.csv', index=False)
insomnia_test_xFeat.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_insomnia_test_xFeat.csv', index=False)
insomnia_test_y.to_csv('/Users/JasonLebov_1/Desktop/CS 470/Final Project/Final Data/Binary Class/binary_insomnia_test_y.csv', index=False)

Visualizations

In [ ]:
# Graph of Pearson Correlation Matrices - Features
plt.figure(figsize=(35,30)) # 35, 30
plt.title('Pearson Correlation Matrix of Features', fontsize=20)
sns.heatmap(feature_corr, annot=True, cmap=plt.cm.Blues, fmt='.2f')

In [ ]:
# Graph of Pearson Correlation Matrices - Labels

In [ ]:
    # Anxiety
plt.figure(figsize=(35,30))
plt.title('Pearson Correlation Matrix of Features vs Anxiety', fontsize=20)
sns.heatmap(anxiety_corr, annot=True, cmap=plt.cm.Blues, fmt='.2f')

In [ ]:
    # Depression
plt.figure(figsize=(35,30))
plt.title('Pearson Correlation Matrix of Features vs Depression', fontsize=20)
sns.heatmap(depression_corr, annot=True, cmap=plt.cm.Blues, fmt='.2f')

In [ ]:
    # OCD
plt.figure(figsize=(35,30))
plt.title('Pearson Correlation Matrix of Features vs OCD', fontsize=20)
sns.heatmap(ocd_corr, annot=True, cmap=plt.cm.Blues, fmt='.2f')

In [ ]:
    # Insomnia
plt.figure(figsize=(35,30))
plt.title('Pearson Correlation Matrix of Features vs Insomnia', fontsize=20)
sns.heatmap(insomnia_corr, annot=True, cmap=plt.cm.Blues, fmt='.2f')